# CSII 2024 # Exerercise 09: Seperation Principle, LQG, Stability Margins
&copy; 2024 ETH Zurich, Timm Grigat, Suno Dieckmann, Dejan Milojevic; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli


## Description
TODO: Description of the notebook.

To start, run the following cells to install the necessary modules and import the libraries.

In [48]:
%pip install cs2solutions #--force-reinstall

notebookname = "ps09.ipynb"

Note: you may need to restart the kernel to use updated packages.


In [49]:
import os
import sys
import control as ct
import numpy as np

sys.path.append(os.path.join(os.getcwd().strip(notebookname), "utils/"))

from library_tools import library_tf_SISO

np.set_printoptions(suppress=True, precision=3)

### $H^\infty$ norm

In this box we will define the $H^\infty$ norm that we will use to check
the stability of our systems. What the  $H^\infty$ norm essentially does is, 
that it looks for the highest singlular value across all frequencies.
Below you can find the definition of the  $H^\infty$ norm. Feel free to have a 
look.

In [50]:
def hinf_norm(system: ct.TransferFunction) -> int:
    """Calculate the h_inf_norm of a given systems transfer function."""
    # Calculate frequency response over a wide range of frequencies
    omega = np.linspace(-4, 4, 1000)
    H = system(omega * 1j)
    # Calculate all the singular values after checking for MIMO
    if system.ninputs > 1 or system.noutputs > 1:
        singular_values = [
            np.linalg.svd(H[..., i], compute_uv=False) for i in range(len(omega))
        ]
    else:
        singular_values = [np.absolute(H[..., i]) for i in range(len(omega))]
    # Return the highest singular value
    return np.vstack(singular_values).max()

# Exercise 1: Small gain Theorem

Consider the following System with three plants(P0, P1, P2).
Write a function that checks if the cobined system is stable using the small gain theorem.
You have the following functions at hand:
- __hinf_norm(transfer_function)__ which calculates the h infinity norm from a given lti system e.g. _np.hinf_norm(tf: ct.TransferFunction) = __
- __library_tf_SISO(system_number)__ which gives you the transfer function and name of the subsystems e.g. library_tf_SISO(0)[0] for P0.

Also check if all conditions of the small gain theorem are fulfilled.

Your function should return __True__ if all conditions of the small gain theorem are fulfilled and __False__ if not.
YOu may implement the function __is_stable__ that checks whether the given transfer functions are stable or not.

### System
<img src=./images/block_diagram_0.png alt="Image" width="600" height="200"> 

In [51]:
#Define helper functions if you want to use any.

def small_gain_theorem(systems: list[int]) -> bool:
    """Checks if the system is stable depending on the given subsystems."""
    # TODO: Write a function to check for the stability using the small gain theorem.
    return True

small_gain_theorem([0, 1])

In [54]:
def is_stable_solution(systems: list[ct.TransferFunction]) -> bool:
    """Checks if the system is stable depending on the given subsystem."""
    stable =[pole.real < 0 for system in systems for pole in system.poles()]
    return np.all(stable)

def small_gain_theorem_solution(systems: list[int]) -> bool:
    """Checks if the system is stable depending on the given subsystems."""
    # TODO: Write a function to check for the stability using the small gain theorem.
    list_tf = [library_tf_SISO(system)[0] for system in systems]
    if not is_stable_solution(list_tf):
        print("Not all systems are stable, small gain theorem cannot be applied.")
        return False
    list_gamma = [hinf_norm(tf) for tf in list_tf]
    return np.prod(list_gamma) < 1

small_gain_theorem_solution([0, 1])

Not all systems are stable, small gain theorem cannot be applied.


False

# Exercise 2: Stability
This time we want to check for internal stability.
Write a function __internally_stability_check(systems:list[int])__
You can use __library_tf_SISO(system_number)__ jsut as you did in Exercise 1.

__HINT:__ All the systems are SISO, so the inverse becomes rather simple.

<img src=./images/block_diagram_1.png alt="Image" width="600" height="200"> 

In [ ]:
def internal_stability_check_solution(systems: list[int]) -> bool:
    """Checks the complete internal stability of the system."""
    # TODO: Write a function that checks the internal and external stability of the system.
    #define all four transfer functions
    return True

internal_stability_check_solution([0,1,2])

In [57]:
def internal_stability_check(systems: list[int]) -> bool:
    """Checks the complete internal stability of the system."""
    # TODO: Write a function that checks the internal and external stability of the system.
    #define all four transfer functions
    list_tf = [library_tf_SISO(system)[0] for system in systems]
    P0, P1, P2 = list_tf[0], list_tf[1], list_tf[2]
    P01 = P0*P1
    I_P01_P2_inv = 1/(1-P01*P1)    #This inverse is only possible due to the fact that we are using SISO systems
    I_P2_P01_inv = 1/(1-P1*P01)
    closed_loop_tfs = [I_P2_P01_inv,I_P2_P01_inv*P2,I_P01_P2_inv,I_P01_P2_inv*P2]

    stable =[pole.real < 0 for tf in closed_loop_tfs for pole in tf.poles()]
    return np.all(stable)

internal_stability_check([0,1,2])

True

# Exercise 3: Performance Robustness
Think of a cool exercise.